# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [6]:
!pip install mxnet-cu92
!pip install d2l
import mxnet as mx 
from mxnet import ndarray as nd

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [0]:
import time
tic= time.time()
A = mx.nd.ones((4096,4096))
B = mx.nd.ones((4096,4096))
C = mx.nd.dot(A,B)
print(time.time() - tic) 

C.wait_to_read()
print(time.time() - tic) 

A.astype(vector, copy=True)
B.astype(vector, copy=True)
tic2= time.time()

#GPU should be able to mine faster than a CPU since it specializes in these image display functions. 

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

A symmetric matrix is a positive semidefinite matrix if and only if all eigenvalues are non-negative. D is orthogonal with non-negative entries as stated. With A as an arbitrary vector, the resulting matrix contains n orthogonal eigenvectors of A, which has the condition that its eigenvalues are non-negative. Therefore, B is a positive semidefinite matrix.
It would be useful to work with B when working with multiple constraints or multiple dimensions. The individual vectors would be more useful when manipulating the variables directly or trying to identify single vector components.

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

In [11]:

from mxnet.gluon import nn

mx.cpu(), mx.gpu(), mx.gpu(1)


g= nd.random.uniform(shape=(2, 2), ctx=mx.gpu(1))
print(g)


MXNetError: ignored

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [8]:
import time

A = nd.ones((4096,4096))
B = nd.ones((4096,4096))
c1= nd.norm(nd.dot(A, B))
c= mx.ndarray.power(c1,2)
tic= time.time()
print(time.time() - tic) 

d= np.array(time.time() - tic) 
c= d.asnumpy(d.copy())

NameError: ignored

## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [0]:
a = nd.random.norm(0,1, shape = (5,5))
b = nd.random.norm(0,1, shape = (5,5))
c = nd.random.norm(0,1, shape = (5,5))

c[:] = a * b + c
print("id(a):", id(a))
print("id(b):", id(b))
print("id(c):", id(c))

NameError: name 'A' is not defined

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [0]:
import numpy as np

i = 10
j = 20

ir = np.zeros((i, j))
ir[0, :] = [1]
ir = ir.ravel()

mat = []
for i in range(i):
    mat.append(ir)
    ir = np.roll(ir, j)

ic = np.zeros((i, j))
ic[:, 0] = 1
ic = ic.ravel()

for i in range(j):
    mat.append(ic)
    ic = np.roll(ic, i)

mat = np.asarray(mat).T